# sklearn-LDA

代码示例：https://mp.weixin.qq.com/s/hMcJtB3Lss1NBalXRTGZlQ （玉树芝兰） <br>
可视化：https://blog.csdn.net/qq_39496504/article/details/107125284  <br>
sklearn lda参数解读:https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
<br>中文版参数解读：https://blog.csdn.net/TiffanyRabbit/article/details/76445909
<br>LDA原理-视频版：https://www.bilibili.com/video/BV1t54y127U8
<br>LDA原理-文字版：https://www.jianshu.com/p/5c510694c07e
<br>score的计算方法：https://github.com/scikit-learn/scikit-learn/blob/844b4be24d20fc42cc13b957374c718956a0db39/sklearn/decomposition/_lda.py#L729
<br>主题困惑度1：https://blog.csdn.net/weixin_43343486/article/details/109255165
<br>主题困惑度2：https://blog.csdn.net/weixin_39676021/article/details/112187210

## 1.预处理

In [1]:
import os
import pandas as pd
import re
import jieba
import jieba.posseg as psg

In [2]:
# output_path = 'D:/python/lda/result'
# file_path = 'D:/python/lda/data'
# os.chdir(file_path)
data=pd.read_excel("data.xlsx")#content type
# os.chdir(output_path)
dic_file = "dict.txt"
stop_file = "stopwords.txt"
data

,content,type
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军\n 记者傅亚雨沈阳报道 来到沈阳，国奥队...,体育
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典\n 多曼来了，瑞典来了，商瑞华首战求3分...,体育
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛\n 新浪体育讯 12月27日晚，“冠军高尔...,体育
3,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀\n 新浪体育讯 2月24日，辽足爆发了集...,体育
4,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹\n 体坛周报特约记者张锐北京报道 谢亚龙已...,体育
...,...,...
795,12月9日美股专家坐堂实录\n 12月9日，美股专家warren王做客新浪财经，回答新浪网...,股票
796,12月10日美股专家坐堂实录\n 12月10日，美股专家warren王做客新浪财经，回答新...,股票
797,12月15日美股专家坐堂实录\n 12月15日，美股专家Aries高做客新浪财经，回答新浪...,股票
798,12月17日美股专家坐堂实录\n 12月17日，美股专家Aries高做客新浪财经，回答新浪...,股票


In [3]:
def chinese_word_cut(mytext):
    jieba.load_userdict(dic_file)
    jieba.initialize()
    try:
        stopword_list = open(stop_file,encoding ='utf-8')
    except:
        stopword_list = []
        print("error in stop_file")
    stop_list = []
    flag_list = ['n','nz','vn']
    for line in stopword_list:
        line = re.sub(u'\n|\\r', '', line)
        stop_list.append(line)
    
    word_list = []
    #jieba分词
    seg_list = psg.cut(mytext)
    for seg_word in seg_list:
        word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        #word = seg_word.word  #如果想要分析英语文本，注释这行代码，启动下行代码
        find = 0
        for stop_word in stop_list:
            if stop_word == word or len(word)<2:     #this word is stopword
                    find = 1
                    break
        if find == 0 and seg_word.flag in flag_list:
            word_list.append(word)      
    return (" ").join(word_list)

In [4]:
data["content_cutted"] = data.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3c/v18vsqt50jzgnmgzv858z_rm0000gn/T/jieba.cache
Loading model cost 0.487 seconds.
Prefix dict has been built successfully.


## 2.LDA分析

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [7]:
n_features = 1000 #提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [8]:
n_topics = 8
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=8,
                          random_state=0)

### 2.1输出每个主题对应词语 

In [9]:
n_top_words = 25
tf_feature_names = tf_vectorizer.get_feature_names()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

### 2.2输出每篇文章对应主题 

In [ ]:
import numpy as np

In [ ]:
topics=lda.transform(tf)

In [ ]:
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
data['概率最大的主题序号']=topic
data['每个主题对应概率']=list(topics)
data.to_excel("data_topic.xlsx",index=False)

### 2.3可视化 

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()
pic = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(pic)
pyLDAvis.save_html(pic, 'lda_pass'+str(n_topics)+'.html')
pyLDAvis.display(pic)
#去工作路径下找保存好的html文件
#和视频里讲的不一样，目前这个代码不需要手动中断运行，可以快速出结果

### 2.4困惑度 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plexs = []
scores = []
n_max_topics = 16
for i in range(1,n_max_topics):
    print(i)
    lda = LatentDirichletAllocation(n_components=i, max_iter=50,
                                    learning_method='batch',
                                    learning_offset=50,random_state=0)
    lda.fit(tf)
    plexs.append(lda.perplexity(tf))
    scores.append(lda.score(tf))

In [ ]:
n_t=15#区间最右侧的值。注意：不能大于n_max_topics
x=list(range(1,n_t+1))
plt.plot(x,plexs[0:n_t])
plt.xlabel("number of topics")
plt.ylabel("perplexity")
plt.show()